# Python Homework
-- Tuğçe Kul




Library importing and init some varibles for later usage 

In [20]:
import pandas as pd  # Import pandas library
import statistics  # Import statistics library
import numpy as np

ORDER = "Order"
BRUT_M2 = "Brüt m²"
NET_M2 = "Net m²"
ROOMS = "Rooms"
BATH = "Bath"
FLOORS = "Floors"
FLOOR_LOCATION = "Floor Location"
BUILDING_AGE = "Building Age"
HEATING_TYPE = "Heating Type"
FURNITURE = "Furniture"
USING_STATUS = "Using Status"
BUILDING_STATUS = "Building Status"
HOUSING_TYPE = "Housing Type"
RENT_EARNING = "Rent Earning"
FROM = "From"
SUITABLE_FOR_CREDIT = "Suitable for Credit"
PRICE = "Price"
COORD1 = "Coord1"
COORD2 = "Coord2"
STREET = "Street"
DISTRICT = "District"
LOCATION3 = "Location3"
DUE_PRICE = "Due Price"
SUITABLE_FOR_EXCHANGE = "Suitable for exchange"

FILLER_TYPE = {
    ORDER: "continuous",
    BRUT_M2: "continuous",
    NET_M2: "continuous",
    ROOMS: "continuous",
    BATH: "continuous",
    FLOORS: "categorical",
    FLOOR_LOCATION: "categorical",
    BUILDING_AGE: "continuous",
    HEATING_TYPE: "categorical",
    FURNITURE: "categorical",
    USING_STATUS: "categorical",
    BUILDING_STATUS: "categorical",
    HOUSING_TYPE: "categorical",
    RENT_EARNING: "continuous",
    FROM: "categorical",
    SUITABLE_FOR_CREDIT: "categorical",
    PRICE: "continuous",
    COORD1: "continuous",
    COORD2: "continuous",
    STREET: "categorical",
    DISTRICT: "categorical",
    LOCATION3: "categorical",
    DUE_PRICE: "continuous",
    SUITABLE_FOR_EXCHANGE: "categorical",
}

1. Import the data as a dataframe.

In [21]:
df = pd.read_csv("data.csv")

<code>get_all_districts</code> function is gonna be very useful later. The function returns all unique districts in df.

In [56]:
def get_all_districts():
    return df["District"].unique()

2. <code>fill_missing_prices</code> function iterate all districts and finds the average price of it and place into df as integer.

In [57]:
def find_average_price(district):
    prices = df[df["District"] == district]["Price"]  # Get all prices of district
    prices = prices[prices.notnull()]  # Remove NaN values
    return int(prices.mean())  # Return average price of district


def fill_missing_prices():
    for district in get_all_districts():  # Iterate over all districts
        avg_price = find_average_price(district)  # Find average price of district
        df.loc[df["District"] == district, "Price"] = df.loc[
            df["District"] == district, "Price"
        ].fillna(
            avg_price
        )  # Fill missing prices with average price of district
    df["Price"] = df["Price"].astype(int)  # Convert Price column to int type

3. <code>fill_empty_columns</code> function is takes a column_name that in FILLER_TYPE and from FILLER_TYPE gets its type. if type continuous gets the median of it, if type is categorical get its mode.

In [58]:
def fill_empty_columns(column_name):
    if FILLER_TYPE[column_name] == "continuous":  # If column is continuous
        # Fill missing values with median
        df[column_name] = df[column_name].fillna(
            type(df[column_name]) == int
            and df[column_name].median()
            or type(df[column_name]) == float
            and df[column_name].median()
        )
    elif FILLER_TYPE[column_name] == "categorical":  # If column is categorical
        df[column_name] = df[column_name].fillna(
            df[column_name].mode()[0]
        )

4. <code>find_and_remove_duplicates</code> function first sort everything by price. Low price is first now. Then if everything same other than price, drop it and keep the first one (a.k.a lowest price). Then sort by order again to back to normal.

In [59]:
def find_and_remove_duplicates():
    df.sort_values(by=[PRICE], inplace=True, ascending=True) # Sort by price ascending
    df.drop_duplicates(  # Drop duplicates
        subset=[
            BRUT_M2,
            NET_M2,
            ROOMS,
            BATH,
            FLOORS,
            FLOOR_LOCATION,
            BUILDING_AGE,
            HEATING_TYPE,
            FURNITURE,
            USING_STATUS,
            BUILDING_STATUS,
            HOUSING_TYPE,
            RENT_EARNING,
            FROM,
            SUITABLE_FOR_CREDIT,
            COORD1,
            COORD2,
            STREET,
            DISTRICT,
            LOCATION3,
            DUE_PRICE,
            SUITABLE_FOR_EXCHANGE,
        ],
        keep="first",  # Keep first
        inplace=True,
    )
    df.sort_values(by=[ORDER], inplace=True)  # Sort by order

5. <code>create_new_df</code> function creates new dataframe from first df. Only have two columns "District" and "Average Price". First we will create a new df from columns. Iterate over all district and find the average price of the districts. Then create the data from it. Lastly add the data to new df then return it.

In [60]:
def create_new_df():
    columns = ["District", "Average Price"]
    new_df = pd.DataFrame(columns=columns) # Create new dataframe
    for district in get_all_districts(): # Iterate over all districts
        avg_price = find_average_price(district) # Find average price of district
        new_data = {"District": district, "Average Price": avg_price} # Create new data
        new_df = new_df._append(new_data, ignore_index=True) # Append new data to new dataframe
    return new_df # Return new dataframe

6. <code>add_std_to_new_df</code> function adds the STD to new column named "Standard Deviation". <code>find_std</code> function gets all prices of district and removes to null values if any. Then build in std() function finds the their std and returns. Then the value that we found puts into "Standard Deviation" column for that row.

In [61]:
def find_std(district):
    prices = df[df["District"] == district]["Price"]  # Get all prices of district
    prices = prices[prices.notnull()]  # Remove NaN values
    return prices.std()  # Return standard deviation of prices in districtd

def add_std_to_new_df(new_df):
    new_df["Standard Deviation"] = new_df["District"].apply(find_std) # Add standard deviation column to new dataframe and apply the find_std function
    return new_df # Return new dataframe

7. 

8. <code>find_the_houses</code> function returns a dictionary that contains the number of houses in each price range. First of all, we create a dictionary with the keys 0-250, 250-500 and 500 and over. Then we iterate over all rows and check the price of the house. If the price is less than 250000, we increase the 0-250 count by 1. If the price is less than 500000, we increase the 250-500 count by 1. If the price is greater than 500000, we increase the 500 and over count by 1. Finally, we return the dictionary.

In [62]:
def find_the_houses():
    ret = {"0-250": 0, "250-500": 0, "500 and over": 0} # Create return dictionary
    for index, row in df.iterrows(): # Iterate over all rows
        if row["Price"] < 250000: # If price is less than 250000
            ret["0-250"] += 1 # Increase 0-250 count
        elif row["Price"] < 500000: # If price is less than 500000
            ret["250-500"] += 1 # Increase 250-500 count
        else: # If price is greater than 500000
            ret["500 and over"] += 1 # Increase 500 and over count
    return ret # Return dictionary

9. <code>find_the_houses_with_given_prices</code> function returns a dictionary that contains the number of houses in each price range. First of all, we create a dictionary with the keys 0-first, first-second and so on. Then we iterate over all rows and check the price of the house. If the price is less than first price, we increase the 0-first count by 1. If the price is less than second price, we increase the first-second count by 1. If the price is greater than last price, we increase the last count by 1. Finally, we return the dictionary.

In [63]:
def find_the_houses_with_given_prices(list_of_prices): # list_of_prices is a list of prices
    if len(list_of_prices) < 2: # If list of prices is less than 2
        return None # Return None
    ret = { # Create return dictionary
        "0-" + str(list_of_prices[0]): 0, # Create first key
    }
    for i in range(1, len(list_of_prices)): # Iterate over list of prices
        ret[str(list_of_prices[i - 1]) + "-" + str(list_of_prices[i])] = 0 # Create key
    ret[str(list_of_prices[len(list_of_prices) - 1]) + " and over"] = 0 # Create last key
    for index, row in df.iterrows(): # Iterate over all rows
        if row["Price"] < list_of_prices[0] * 1000: # If price is less than first price
            ret["0-" + str(list_of_prices[0])] += 1 # Increase 0-first count
        else: # If price is greater than first price
            for i in range(1, len(list_of_prices)): # Iterate over list of prices
                if row["Price"] < list_of_prices[i] * 1000: # If price is less than current price
                    ret[str(list_of_prices[i - 1]) + "-" + str(list_of_prices[i])] += 1 # Increase current count
                    break # Break loop
            else: # If price is greater than last price
                ret[str(list_of_prices[len(list_of_prices) - 1]) + " and over"] += 1 # Increase last count
    return ret # Return dictionary

10. <code>find_the_average_price_of_houses_via_room_count</code> function returns a dictionary that contains the average price of houses with the same room count. First of all, we create a dictionary with the keys 1, 2, 3, 4, 5 and 5+. Then we iterate over all rows and check the room count of the house. If the room count is in the dictionary, we append the price to the list. If the room count is not in the dictionary, we create a new list. Finally, we remove NaN values from the list and calculate the average price. Then we return the dictionary.

In [64]:
def find_the_average_price_of_houses_via_room_count():  # list_of_prices is a list of prices
    ret = {}  # Create return dictionary
    for index, row in df.iterrows():  # Iterate over all rows
        if row["Rooms"] in ret:  # If room count is in dictionary
            ret[row["Rooms"]].append(row["Price"])  # Append price to list
        else:  # If room count is not in dictionary
            ret[row["Rooms"]] = [row["Price"]]  # Create new list
    for key in ret:  # Iterate over all keys
        ret[key] = [
            x for x in ret[key] if not math.isnan(x)
        ] # Remove NaN values from list
        ret[key] = int(statistics.mean(ret[key]))  # Calculate average price
    return ret  # Return dictionary

11. <code>find_the_average_price_of_houses_via_room_count_and_district</code> function returns a dictionary that contains the average price of houses with the same room count and district. First of all, we create a dictionary with the keys 1, 2, 3, 4, 5 and 5+. Then we iterate over all rows and check the room count of the house. If the room count is equal to given room count, we check the district of the house. If the district is in the dictionary, we append the price to the list. If the district is not in the dictionary, we create a new list. Finally, we remove NaN values from the list and calculate the average price. Then we return the dictionary.

In [65]:
def find_the_average_price_of_houses_via_room_count_and_district(room="3+1"):
    ret = {} # Create return dictionary
    for index, row in df.iterrows(): # Iterate over all rows
        if row["Rooms"] == room: # If room count is equal to given room count
            if row["District"] in ret: # If district is in dictionary
                ret[row["District"]].append(row["Price"]) # Append price to list
            else: # If district is not in dictionary
                ret[row["District"]] = [row["Price"]] # Create new list
    for key in ret: # Iterate over all keys
        ret[key] = [
            x for x in ret[key] if not math.isnan(x)
        ] # Remove NaN values from list
        ret[key] = int(statistics.mean(ret[key])) # Calculate average price
    return ret # Return dictionary

12. <code>print_ordered_df_of_house_prices</code> function prints the dataframe ordered by price. First of all, we copy the dataframe. Then we sort the dataframe by price. Finally, we print the dataframe.

In [66]:
def print_ordered_df_of_house_prices():
    df2 = df.copy()  # Copy dataframe
    df2 = df2.sort_values(by=[PRICE])  # Sort by price
    print(df2)  # Print dataframe

13. <code>print_ordered_df</code> function prints the dataframe ordered by district and street. First of all, we copy the dataframe. Then we normalize the street names (i.e. İ -> I). Then we decode the street names. Then we sort the dataframe by district and street. Finally, we print the dataframe.

In [67]:
def print_ordered_df():
    df2 = df.copy() # Copy dataframe
    df2["Street"] = (
        df2["Street"].str.normalize("NFKD").str.encode("ascii", errors="ignore")
    ) # Normalize street names (i.e. İ -> I)
    df2["Street"] = df2["Street"].str.decode("utf-8") # Decode street names
    df2 = df2.sort_values(by=["District", "Street"]) # Sort by district and street
    print(df2) # Print dataframe

14. </code>scatter_plot</code> function returns a scatter plot of net m2 and price. First of all, we create a scatter plot of net m2 (X) and price (Y). Then we return the scatter plot.

In [68]:
def scatter_plot():
    return df.plot.scatter(x=NET_M2, y=PRICE)

15. <code>add_price_per_m2</code> function adds a new column (Price per m2) to the dataframe. First of all, we divide the price by net m2. Then we round the result to 2 decimal places. Finally, we return the dataframe.

In [69]:
def add_price_per_m2():
    df["Price per m2"] = (df[PRICE] / df[NET_M2]).round(2)
    return df

16. <code>add_average_price_per_m2_to_district_df</code> function returns a new dataframe. First of all, we create a new dataframe. Then we add the price per m2 column to the dataframe. Then we add the average price per m2 column to the dataframe by appling <code>find_average_price_per_m2</code>. <code>find_average_price_per_m2</code> function returns the average price per m2 of the given district. First of all, we get all prices of the district. Then we remove NaN values. Finally, we return the average price of the district.
Finally return the last df.

In [70]:
def find_average_price_per_m2(district):
    prices = df[df[DISTRICT] == district]["Price per m2"]  # Get all prices of district
    prices = prices[prices.notnull()]  # Remove NaN values
    return int(prices.mean())  # Return average price of district


def add_average_price_per_m2_to_district_df():
    district_df = create_new_df()  # Create dictrict dataframe
    add_price_per_m2()  # Add price per m2 column to dataframe
    district_df["Average Price per m2"] = district_df["District"].apply(
        find_average_price_per_m2
    )  # Add average price per m2 column to new dataframe
    return district_df # Return new dataframe

17. <code>bar_plot</code> function returns a bar plot of average price per m2. First of all, we create a new dataframe. Then we sort the dataframe by average price per m2. Finally, we return the bar plot.

In [71]:
def bar_plot():
    district_df = add_average_price_per_m2_to_district_df()  # Create dictrict dataframe
    district_df = district_df.sort_values(
        by=["Average Price per m2"], ascending=False
    )  # Sort by average price per m2
    return district_df.plot.bar(x=DISTRICT, y="Average Price per m2")

18. <code>add_price_predictions</code> function adds a new column to the dataframe. The new column is the price prediction. First of all, we add the price prediction column to the dataframe by using <code>find_3_nearest_houses_average_price</code>. <code>find_3_nearest_houses_average_price</code> function returns the average price of the 3 nearest houses. First of all, we calculate the distance between the house and all other houses. Then we find the 3 nearest houses. Then we drop the distance column. Finally, we calculate the average price of the 3 nearest houses and return it. Then we return the dataframe.

In [72]:
def find_3_nearest_houses_average_price(x1, y1):
    df["Distance"] = np.sqrt(
        (df[COORD1] - x1) ** 2 + (df[COORD2] - y1) ** 2
    )  # Calculate distance
    nearest_houses = df.nsmallest(3, "Distance")  # Find 3 nearest houses
    df.drop("Distance", axis=1, inplace=True)  # Drop distance column
    price = int(nearest_houses[PRICE].mean())  # Calculate average price
    return price  # Return average price


def add_price_predictions():
    df["Price Prediction"] = df.apply(
        lambda row: find_3_nearest_houses_average_price(row[COORD1], row[COORD2]),
        axis=1,
    )  # Add price prediction column to dataframe
    return df  # Return dataframe